In [1]:
import os, csv, math
import pandas as pd
import numpy as np

channels_per_grid = 32
grid_length = 8
grid_width = 4


In [2]:

def is_csv_66_columns(csv_file_path):
    with open(csv_file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if len(row) != 66:
                return False
    return True


def is_string_present_in_csv(csv_file_path, search_string):
    
    with open(csv_file_path, 'r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        participants_column = 'Participants'

        for row in reader:
            if search_string in row[participants_column]:
                return True

    return False


def create_or_empty_file(file_path):
    
    if os.path.exists(file_path):
        with open(file_path, 'w') as file:
            file.truncate(0)
        print(f"File '{file_path}' existed and has been emptied.")
    else:
        with open(file_path, 'w') as file:
            pass
        print(f"File '{file_path}' has been created.")
        

def calculate_average(lst):
    if not lst:
        return None

    total = sum(lst)
    average = total / len(lst)
    return average


def get_filenames_with_keywords(folder_path, keywords):
    filenames = []
    for filename in os.listdir(folder_path):
        if any(keyword in filename for keyword in keywords):
            filenames.append(filename)
    return sorted(filenames)


def read_csv_as_list(csv_file):
    data = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            data.append(row)
    return data


def calculate_entropy(data):
    data_rms = np.array(data)
    rms_sq = data_rms ** 2
    sum_rms_sq = np.sum(rms_sq)
    probability = rms_sq / sum_rms_sq
    entropy = -np.sum(probability * np.log2(probability))
    return entropy


def calculate_coefficient_of_variation(signal):
    mean = np.mean(signal)
    std = np.std(signal)
    coefficient_of_variation = (std / mean) * 100
    return coefficient_of_variation


def calculate_intensity(signal):
    
    if(np.sum(signal) > 0):
        intensity = math.log(np.sum(signal)/channels_per_grid, 10)
        return intensity
    else:
        return 0


def calculate_differential_intensity(csv_file, row_index, g1, g2, column_start, column_end):
    
    emg_data = np.genfromtxt(csv_file, delimiter=',')
    emg_row = emg_data[int(row_index), column_start:column_end]
    
    if column_start==0:
        divisor = float(g1)
    else:
        divisor = float(g2)
    
    emg_row_data = [x/divisor for x in emg_row]
    
    emg_diff_data = []
    
    for i in range(1,len(emg_row_data)):
        emg_diff_data.append(math.log(abs(emg_row_data[i] - emg_row_data[i-1]),10))
        
    return calculate_average(emg_diff_data)


def calculate_mean(signal):
    mean = np.mean(signal)
    return mean


def calculate_max(signal):
    max_value = np.max(signal)
    return max_value


def calculate_center_of_gravity(csv_file, row_index, column_start, column_end):
    
    emg_data = np.genfromtxt(csv_file, delimiter=',')
    emg_row_data = emg_data[row_index, column_start:column_end] 

    # Reshape the 1D array into a 2D grid (8x4)
    emg_grid = emg_row_data.reshape(grid_length, grid_width)

    # Calculate the CoG X and Y coordinates
    row_indices, col_indices = np.indices((grid_length, grid_width))
    total_rms = np.sum(emg_grid)
    cog_x = np.sum(col_indices * emg_grid) / total_rms
    cog_y = np.sum(row_indices * emg_grid) / total_rms

    return cog_x, cog_y



def calculate_displacement(xcg1, ycg1, xcg2, ycg2):
    displacement = ((xcg2 - xcg1) ** 2 + (ycg2 - ycg1) ** 2) ** 0.5
    return displacement



def get_normalization_values(target_filename):
    
    target_filename = target_filename[:-4]
    input_dataframe = pd.read_csv("MVC_Data.csv")
    #print(input_dataframe)
    participant_info = target_filename.split("_")
    filtered_rows_E = input_dataframe[(input_dataframe['Participants'] == participant_info[0]) & (input_dataframe['Arm'] == participant_info[1]) & (input_dataframe['Exercise type'] == "Extension")]
    filtered_rows_F = input_dataframe[(input_dataframe['Participants'] == participant_info[0]) & (input_dataframe['Arm'] == participant_info[1]) & (input_dataframe['Exercise type'] == "Flexion")]    
        
    g1_E = filtered_rows_E['G1 RMS Value']
    g2_E = filtered_rows_E['G2 RMS Value']
    g1_F = filtered_rows_F['G1 RMS Value']
    g2_F = filtered_rows_F['G2 RMS Value']
    
    return float(g1_E), float(g2_E), float(g1_F), float(g2_F)



def get_FE_peak_indexes(csv_file_path, search_string):
    
    with open(csv_file_path, 'r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        filename_column = 'Filename'
        extension_column = 'Extension'
        flexion_column = 'Flexion'

        for row in reader:
            if row[filename_column] == search_string:
                extension_value = row[extension_column]
                flexion_value = row[flexion_column]
                return extension_value, flexion_value

    # If the search string is not found, return None for both values
    return None, None

    
    
def compute_EMG_parameters(csv_file, operation, target_row_index, g1, g2, column_start, column_end):

    data = read_csv_as_list(csv_file)
    element_list = []
    
    for col in range(column_start, column_end):
        
        if col < 33:
            normalization_factor = g1
        else:
            normalization_factor = g2
                
        if "MDF" in csv_file:
            normalization_factor = 1
            
        if "ARV" in csv_file:
            normalization_factor = 1
        
        element = float(data[target_row_index][col])
        element_list.append(element/float(normalization_factor))
        
    #print("Operation : ", operation)
    #print(column_start, column_end)
    #print("Row Index : ", target_row_index)
    #print("Normalization Factor : ", normalization_factor)
    #print("Element List : ", element_list)
    
    return operation(element_list)



In [3]:
labels = ['Participant', 'Arm', 'Exercise type', 'Flexion/Extension', 'Entropy_1', 'CoV_1', 'Intensity_1', 'Differential Intensity_1', 'Mean_RMS_1', 'Max_RMS_1', 'Mean_ARV_1', 'Max_ARV_1','Xcg_1', 'Ycg_1','Entropy_2', 'CoV_2', 'Intensity_2', 'Differential Intensity_2', 'Mean_RMS_2', 'Max_RMS_2','Mean_ARV_2', 'Max_ARV_2', 'Xcg_2', 'Ycg_2']
Data_list = []
        
folder_path = './EMG_RMS_Data'
folder_path_mdf = './EMG_MDF_Data'
folder_path_arv = './EMG_ARV_Data'

keywords = ["60", "90", "120"]
matching_filenames = get_filenames_with_keywords(folder_path, keywords)


for filename in matching_filenames:
    
    if(is_string_present_in_csv("./MVC_Data.csv",filename[0:3]) and is_csv_66_columns(folder_path+"/"+filename)):
    
        print(filename)
        participant_list = filename.split("_")
        g1_E, g2_E, g1_F, g2_F = get_normalization_values(filename)

        peak_E, peak_F = get_FE_peak_indexes("./Non_Functional_Indexes.csv",filename)
        peak_E = int(peak_E)
        peak_F = int(peak_F)
        peak_E = math.floor(peak_E/10)
        peak_F = math.floor(peak_F/10)
        csv_file = folder_path + "/" + filename
        csv_file_mdf = folder_path_mdf + "/" + filename
        csv_file_arv = folder_path_arv + "/" + filename
        #print("g1_F : ",g1_F)
        #print("g1_E : ",g1_E)
        #print("g2_F : ",g2_F)
        #print("g2_E : ",g2_E)

        participant_list_extension = participant_list.copy()
        participant_list_extension.append("Extension")

        participant_list_flexion = participant_list.copy()
        participant_list_flexion.append("Flexion")

        participant_list_extension.append(compute_EMG_parameters(csv_file, calculate_entropy, peak_E, g1_E, g2_E, 1, 33))
        participant_list_extension.append(compute_EMG_parameters(csv_file, calculate_coefficient_of_variation, peak_E, g1_E, g2_E, 1, 33))
        participant_list_extension.append(compute_EMG_parameters(csv_file, calculate_intensity, peak_E, g1_E, g2_E, 1, 33))
        participant_list_extension.append(calculate_differential_intensity(csv_file, peak_E, g1_E, g2_E, 1, 33))
        participant_list_extension.append(compute_EMG_parameters(csv_file, calculate_mean, peak_E, g1_E, g2_E, 1, 33))
        participant_list_extension.append(compute_EMG_parameters(csv_file, calculate_max, peak_E, g1_E, g2_E, 1, 33))
        #participant_list_extension.append(compute_EMG_parameters(csv_file_mdf, calculate_mean, peak_E, g1_E, g2_E, 1, 33))
        #participant_list_extension.append(compute_EMG_parameters(csv_file_mdf, calculate_max, peak_E, g1_E, g2_E, 1, 33))
        participant_list_extension.append(compute_EMG_parameters(csv_file_arv, calculate_mean, peak_E, g1_E, g2_E, 1, 33))
        participant_list_extension.append(compute_EMG_parameters(csv_file_arv, calculate_max, peak_E, g1_E, g2_E, 1, 33))
        cogx_g1, cogy_g1 = calculate_center_of_gravity(csv_file, int(peak_E), 1, 33)
        participant_list_extension.append(cogx_g1)
        participant_list_extension.append(cogy_g1)

        participant_list_extension.append(compute_EMG_parameters(csv_file, calculate_entropy, peak_E, g1_E, g2_E, 33, 65))
        participant_list_extension.append(compute_EMG_parameters(csv_file, calculate_coefficient_of_variation, peak_E, g1_E, g2_E, 33, 65))
        participant_list_extension.append(compute_EMG_parameters(csv_file, calculate_intensity, peak_E, g1_E, g2_E, 33, 65))
        participant_list_extension.append(calculate_differential_intensity(csv_file, peak_E, g1_E, g2_E, 33, 65))
        participant_list_extension.append(compute_EMG_parameters(csv_file, calculate_mean, peak_E, g1_E, g2_E, 33, 65))
        participant_list_extension.append(compute_EMG_parameters(csv_file, calculate_max, peak_E, g1_E, g2_E, 33, 65))
        #participant_list_extension.append(compute_EMG_parameters(csv_file_mdf, calculate_mean, peak_E, g1_E, g2_E, 33, 65))
        #participant_list_extension.append(compute_EMG_parameters(csv_file_mdf, calculate_max, peak_E, g1_E, g2_E, 33, 65))
        participant_list_extension.append(compute_EMG_parameters(csv_file_arv, calculate_mean, peak_E, g1_E, g2_E, 33, 65))
        participant_list_extension.append(compute_EMG_parameters(csv_file_arv, calculate_max, peak_E, g1_E, g2_E, 33, 65))
        cogx_g2, cogy_g2 = calculate_center_of_gravity(csv_file, int(peak_E), 33, 65)
        participant_list_extension.append(cogx_g2)
        participant_list_extension.append(cogy_g2)

        participant_list_flexion.append(compute_EMG_parameters(csv_file, calculate_entropy, peak_F, g1_F, g2_F, 1, 33))
        participant_list_flexion.append(compute_EMG_parameters(csv_file, calculate_coefficient_of_variation, peak_F, g1_F, g2_F, 1, 33))
        participant_list_flexion.append(compute_EMG_parameters(csv_file, calculate_intensity, peak_F, g1_F, g2_F, 1, 33))
        participant_list_flexion.append(calculate_differential_intensity(csv_file, peak_F, g1_F, g2_F, 1, 33))
        participant_list_flexion.append(compute_EMG_parameters(csv_file, calculate_mean, peak_F, g1_F, g2_F, 1, 33))
        participant_list_flexion.append(compute_EMG_parameters(csv_file, calculate_max, peak_F, g1_F, g2_F, 1, 33))
        #participant_list_flexion.append(compute_EMG_parameters(csv_file_mdf, calculate_mean, peak_F, g1_F, g2_F, 1, 33))
        #participant_list_flexion.append(compute_EMG_parameters(csv_file_mdf, calculate_max, peak_F, g1_F, g2_F, 1, 33))
        participant_list_flexion.append(compute_EMG_parameters(csv_file_arv, calculate_mean, peak_F, g1_F, g2_F, 1, 33))
        participant_list_flexion.append(compute_EMG_parameters(csv_file_arv, calculate_max, peak_F, g1_F, g2_F, 1, 33))
        cogx_g1, cogy_g1 = calculate_center_of_gravity(csv_file, int(peak_F), 1, 33)
        participant_list_flexion.append(cogx_g1)
        participant_list_flexion.append(cogy_g1)

        participant_list_flexion.append(compute_EMG_parameters(csv_file, calculate_entropy, peak_F, g1_F, g2_F, 33, 65))
        participant_list_flexion.append(compute_EMG_parameters(csv_file, calculate_coefficient_of_variation, peak_F, g1_F, g2_F, 33, 65))
        participant_list_flexion.append(compute_EMG_parameters(csv_file, calculate_intensity, peak_F, g1_F, g2_F, 33, 65))
        participant_list_flexion.append(calculate_differential_intensity(csv_file, peak_F, g1_F, g2_F, 33, 65))
        participant_list_flexion.append(compute_EMG_parameters(csv_file, calculate_mean, peak_F, g1_F, g2_F, 33, 65))
        participant_list_flexion.append(compute_EMG_parameters(csv_file, calculate_max, peak_F, g1_F, g2_F, 33, 65))
        #participant_list_flexion.append(compute_EMG_parameters(csv_file_mdf, calculate_mean, peak_F, g1_F, g2_F, 33, 65))
        #participant_list_flexion.append(compute_EMG_parameters(csv_file_mdf, calculate_max, peak_F, g1_F, g2_F, 33, 65))
        participant_list_flexion.append(compute_EMG_parameters(csv_file_arv, calculate_mean, peak_F, g1_F, g2_F, 33, 65))
        participant_list_flexion.append(compute_EMG_parameters(csv_file_arv, calculate_max, peak_F, g1_F, g2_F, 33, 65))
        cogx_g2, cogy_g2 = calculate_center_of_gravity(csv_file, int(peak_F), 33, 65)
        participant_list_flexion.append(cogx_g2)
        participant_list_flexion.append(cogy_g2)
        
        print(participant_list_extension)
        print(participant_list_flexion)
        
        #print(len(participant_list_extension))
        #print(len(participant_list_flexion))

        Data_list.append(participant_list_extension)
        Data_list.append(participant_list_flexion)
        

df = pd.DataFrame(Data_list, columns=labels)

csv_filename = 'Data_Non_Functional.csv'
create_or_empty_file(csv_filename)
df.to_csv(csv_filename, index=False) 


P01_Left_120.csv
['P01', 'Left', '120.csv', 'Extension', 4.989920498862942, 5.924474715095558, -0.21944376604224547, -2.1713591713497356, 0.6033318243690071, 0.6750081989456892, 363.44701697578944, 406.3758428143851, 1.4657042415699466, 3.4814623508964107, 4.955244133763664, 12.509226125117676, -0.35003829958778515, -1.747432336890574, 0.4466442017284893, 0.5494542890335333, 756.7399383262733, 949.8964034367854, 1.4989720791263161, 3.6924304101482375]
['P01', 'Left', '120.csv', 'Flexion', 4.851426624103005, 23.2984349654059, -0.3762516076890805, -0.8372166553484608, 0.4204829517993136, 0.6102402592287782, 969.4619649364195, 1396.9561835241286, 1.4638419274863506, 3.208279224762251, 4.956243205054518, 12.403817516415694, -0.24659217417578405, -1.4649699546372046, 0.5667712678562093, 0.6918971931458866, 359.97211765858356, 435.5808990510677, 1.4381925240307578, 3.427545472266734]
P01_Left_90.csv
['P01', 'Left', '90.csv', 'Extension', 4.990755436207554, 5.680983923781638, -0.2322265878305

['P02', 'Right', '90.csv', 'Extension', 4.998635126723872, 2.175158792717013, 0.07547969713917026, -2.205286584731797, 1.1898157050884957, 1.2304947141410312, 913.5171457451828, 941.6256676198377, 1.490635214338773, 3.4937731343949934, 4.963671323316325, 11.313530319238838, -0.044189445500651736, -1.3946309939458847, 0.9032553747842638, 1.0834286836652398, 1134.8373134384483, 1346.1114022145348, 1.5129124224891461, 3.406297343946524]
['P02', 'Right', '90.csv', 'Flexion', 4.974530159671076, 9.505382187208296, -0.1924686410011957, -1.5219647251174553, 0.6419945761847792, 0.7313897404582674, 1134.2027238886633, 1294.5601508163415, 1.4547939622723125, 3.391113216566809, 4.999709097989816, 1.00274696557376, -0.09936883882848375, -2.577576214307015, 0.795483473030498, 0.8140944311991504, 961.711395900297, 986.3340488757294, 1.4981637468739586, 3.5075426520620305]
P03_Left_120.csv
['P03', 'Left', '120.csv', 'Extension', 4.999640209811739, 1.1158361088698026, -0.19636172302947674, -2.927853510

['P04', 'Right', '120.csv', 'Extension', 4.99576167768565, 3.8499220152781124, 0.04525543388592706, -1.9610720764582883, 1.109827377475658, 1.1732038236205526, 300.4573640767853, 322.72627746877276, 1.478926558032071, 3.4978942076041553, 4.949268657666631, 13.334815899301173, -0.03784026746056454, -1.566232684065867, 0.9165575359483047, 1.1457356073551825, 435.2479206238145, 538.1828792468652, 1.5320509034753906, 3.296977532059715]
['P04', 'Right', '120.csv', 'Flexion', 4.9713130712138645, 10.071598759962946, -0.14073281136583554, -0.9954030324448935, 0.72321460615364, 0.8427354134773956, 558.3935037528163, 658.5898230855286, 1.4982401319479655, 3.628706919539565, 4.93153945901906, 15.314874891933364, -0.0653681965447753, -1.296733856896061, 0.8602641076863173, 1.1386033138613387, 212.00858150364485, 267.0248664048496, 1.4224628003442576, 3.4688665171819943]
P04_Right_60.csv
['P04', 'Right', '60.csv', 'Extension', 4.941400888864873, 13.462543643792655, 0.11769681654108524, -1.677824453

['P06', 'Left', '60.csv', 'Extension', 4.9896496561542065, 6.020603513357022, -0.3996240519985744, -2.1526847096205275, 0.39845194213155005, 0.436104473179272, 360.3122792632922, 393.0420920109151, 1.4669932785452506, 3.5033188580660823, 4.988119168704639, 6.443809488126659, -0.44670220811051026, -2.0938165162940927, 0.3575179009898516, 0.3921378161899147, 521.781153288097, 573.6486237393676, 1.505221429733575, 3.5917937811730063]
['P06', 'Left', '60.csv', 'Flexion', 4.972344638114246, 9.946944721537776, -0.3290114284321813, -1.4551282185712313, 0.46880104551269836, 0.5501029748420441, 441.76376515680556, 513.2762391507787, 1.5220033668904456, 3.3686585160207505, 4.994603765201841, 4.34787345878636, -0.36090966209759534, -2.1974822951057074, 0.43560247424111515, 0.47012412894229455, 210.92250599040077, 230.20522310857206, 1.4847754783584852, 3.4379406459592095]
P06_Left_90.csv
['P06', 'Left', '90.csv', 'Extension', 4.995564317793356, 3.9324120846267787, -0.41760667357515885, -2.4591135

['P09', 'Right', '90.csv', 'Extension', 4.9984751349889045, 2.297043108123802, -0.12147179536662932, -2.786714516889787, 0.7560111568068917, 0.7925533397571227, 280.12539559006007, 294.01790936888256, 1.4891616204820777, 3.5178445428843594, 4.913531690359119, 17.975365459034247, -0.29289000422126227, -1.4621930659022384, 0.5094598879880827, 0.6778290858208963, 777.060395849531, 1046.3063313850269, 1.5350326190174441, 3.2823872884201446]
['P09', 'Right', '90.csv', 'Flexion', 4.9171210883140395, 17.300419107114987, -0.22094372684742034, -0.663840230431084, 0.6012516388490481, 0.7762138638060474, 1157.162245887741, 1488.5168703706877, 1.4593338835250675, 3.521015933585412, 4.990047416566837, 5.913042648712412, -0.23097064963409775, -1.9835395424209143, 0.587529057409412, 0.6381808940069968, 280.4084682469208, 307.17886883633344, 1.4791294139109865, 3.5263916458721223]
P10_Left_120.csv
['P10', 'Left', '120.csv', 'Extension', 4.991955767457575, 5.275573498928021, -0.18920370685450089, -2.23

...

['P32', 'Left', '120.csv', 'Extension', 4.939336361291433, 14.586807313178115, -0.36220499500191633, -2.1932072369136173, 0.43430517573245603, 0.5904295629261749, 403.73314476156463, 546.5220125337748, 1.4847362564921467, 3.6610882767966806, 4.851330102662554, 22.97901008748619, -0.19928666821505645, -1.321414161316149, 0.6319945479938873, 0.9307130876303378, 740.5863121581152, 1069.8278316217843, 1.5765914239051173, 3.6887394733965984]
['P32', 'Left', '120.csv', 'Flexion', 4.7971325331140635, 26.912023511278814, -0.27535379416060213, -0.9064367734833113, 0.5304521400217219, 0.8404069958334477, 420.01954118870503, 685.2378908034818, 1.6044764454610034, 3.2579532340785855, 4.9989154188077665, 1.9409317387156695, 0.006443358247341146, -2.0783459858874953, 1.0149469860622482, 1.0427311204563359, 277.0380765602436, 288.79741652536916, 1.4917439008783628, 3.483540685624142]
P32_Left_60.csv
['P32', 'Left', '60.csv', 'Extension', 4.955815685299504, 12.599441857810472, -0.38582066997556896, -2

['P33', 'Right', '60.csv', 'Extension', 4.967202486364111, 10.721444624207516, -0.019842219027828907, -1.5909266360528993, 0.955339601789546, 1.1781679091930901, 415.2601292709479, 502.8985406410671, 1.4502251558197417, 3.3490457943502765, 4.971617145857154, 9.932754012447461, -0.018705219400860083, -1.6208592189004378, 0.9578439944118013, 1.1427436157086408, 612.0063027567866, 728.1783703658091, 1.4714923844890848, 3.3273068315951067]
['P33', 'Right', '60.csv', 'Flexion', 4.811795085126709, 27.111848605022775, 0.12979406136648478, -0.6640681218505621, 1.3483233678644857, 1.9519148526918138, 513.6956358715756, 749.1271161333439, 1.595562717953118, 3.9136419085442076, 4.9861586762705485, 6.931574738211252, 0.05129881933586229, -1.3697849805760127, 1.1253790325009447, 1.2623776950262977, 262.44471070969547, 297.3647241112607, 1.4882872372749272, 3.486550384796476]
P33_Right_90.csv
['P33', 'Right', '90.csv', 'Extension', 4.972155391147898, 9.835666101587037, -0.10425913652961653, -1.84012

['P35', 'Left', '90.csv', 'Extension', 4.999867851070461, 0.6766035019392433, -0.12595632317796324, -2.86508549812066, 0.7482447474017639, 0.7584141608767059, 1625.5542881327874, 1645.0510108555945, 1.498572572635736, 3.501446897521534, 4.9953530689818475, 4.039368507320972, -0.12894601979285109, -1.9467086305395687, 0.7431114964721992, 0.7960742894509703, 1725.5328076234123, 1837.0507081216933, 1.498105500498701, 3.528039530726654]
['P35', 'Left', '90.csv', 'Flexion', 4.99974949298058, 0.9332254547292198, -0.039246938682598896, -2.443661440248864, 0.913593626862087, 0.9263549311454815, 1517.633226659951, 1540.6798127979982, 1.5017017971291313, 3.4881043009280104, 4.9985858857712575, 2.234995049320034, -0.02179393057150416, -2.237707651986518, 0.951055955428689, 0.979723786672164, 1524.8868624371466, 1571.579234623287, 1.5041406673178102, 3.5159684416719004]
P35_Right_120.csv
['P35', 'Right', '120.csv', 'Extension', 4.999532714554257, 1.2721785637576353, -0.1260629252289734, -2.6339996

['P37', 'Left', '120.csv', 'Extension', 4.996344828308055, 3.659219937606649, 0.36005700813803987, -2.1825747748172035, 2.291168385742168, 2.343335760021382, 943.9249347241911, 965.5644960437784, 1.5142760286196324, 3.478899540647756, 4.996346487542603, 3.6076821372497023, 0.23429349358707083, -2.009500073956087, 1.7151159809700955, 1.7617704281315965, 943.8826901128391, 969.3718022346388, 1.5018932630230661, 3.4852248022730494]
['P37', 'Left', '120.csv', 'Flexion', 4.9928109223674895, 5.119062078365174, 0.1921220597682255, -1.5681648226756486, 1.5564030018772796, 1.6378146211904139, 937.7302243820191, 983.9069783126407, 1.521517469025389, 3.457341693765237, 4.996966193399845, 3.2810634513858976, 0.335366739515822, -2.0095527755692713, 2.1645456004244554, 2.2156522991561514, 882.3859541030963, 906.630862210048, 1.500313355618437, 3.47666011458608]
P37_Left_60.csv
['P37', 'Left', '60.csv', 'Extension', 4.992003186066492, 5.35437203612735, 0.4133159326604646, -1.489802496958219, 2.590096

['P38', 'Right', '60.csv', 'Extension', 4.988717101716752, 6.269446022487432, -0.08684178627426128, -2.1341132121655524, 0.8187630096385132, 0.9096577493068361, 548.4169495943618, 605.6589782683961, 1.4752749440671842, 3.504472870836183, 4.964159447976057, 11.28156940735191, -0.2907043109548325, -1.8421219673403548, 0.512030332325741, 0.6063574563021761, 940.4970524185703, 1108.4622049970108, 1.458194387089706, 3.2983484958469163]
['P38', 'Right', '60.csv', 'Flexion', 4.975667032340118, 9.288098469087767, -0.12625377767812848, -1.2460245633494458, 0.7477324393395073, 0.8955498298811456, 747.1678726377661, 891.8000975302174, 1.5171701177183767, 3.5235888016175156, 4.991412142385197, 5.3873124324217185, -0.12958829220050533, -1.8021760588786377, 0.7420133308674994, 0.8945709703605252, 385.38573746253815, 450.16232913016927, 1.506937468657677, 3.5400903534783166]
P38_Right_90.csv
['P38', 'Right', '90.csv', 'Extension', 4.9973819030325775, 3.0165502113551788, -0.25575402221233245, -2.62037